In [20]:
import glob
import os

import pandas as pd
import numpy as np
from lxml import etree

from diligent import diligent

pd.set_option('display.max_columns', 200)

In [2]:
filenames = glob.glob('data/datenexport_20160407/*.xml')

In [3]:
def get_row(root):
    return {
        
    }

def get_rows(filenames):
    for filename in filenames:
        root = etree.fromstring(open(filename).read())
        yield get_row(root)

In [ ]:
import json
from rtree import index

from shapely.geometry import shape, Point

class GeoIndex(object):
    def __init__(self, filename):
        with open(filename) as f:
            js = json.load(f)
        self.idx = index.Index()
        self.features = js['features']
    
        for i, feature in enumerate(self.features):
            polygon = shape(feature['geometry'])
            self.idx.insert(i, polygon.bounds)

    def covers(self, lat, lng):
        point = Point(lng, lat)
        for j in self.idx.intersection(point.coords[0]):
            polygon = shape(self.features[j]['geometry'])
            if point.within(polygon):
                yield self.features[j]['properties']
    
    def get_feature_property(self, lat, lng, prop):
        l = list(self.covers(lat, lng))
        if l:
            return l[0][prop]
        return None
    
laender_index = GeoIndex('geodata/bundeslaender.geojson')
kreise_index = GeoIndex('geodata/landkreise.geojson')
assign_geofeature = lambda idx, key: lambda x: idx.get_feature_property(x.lat, x.lng, key)

In [12]:
def get_nursinghome_csv(filename='data/nursinghomes.csv'):
    if os.path.exists(filename):
        df = pd.read_csv(filename)
        df['letzte Aktualisierung durch Pflegeeinrichtung'] = pd.to_datetime(df['letzte Aktualisierung durch Pflegeeinrichtung'])
        return df

    df = pd.read_csv('datenexport_20160407/correctivExport.csv', encoding='latin1', sep=';')

    split_indicator = ['Minimum/Maximum', 'Gesamtpreis/Anteil Pflegekasse/Eigenanteil', 'Gesamtpreis/Anteil Pflegekasse']

    cost_columns = [c for c in df.columns if any([s in c for s in split_indicator])]


    def split_cost_column(column):
        def _split_cost_column(row):
            if pd.isnull(row[column]):
                return row
            amounts = row[column].replace('.', '').replace(',', '.').split('/')
            parts = column.split('/')
            name, first_part = parts[0].rsplit(' ', 1)
            name_parts = [first_part] + parts[1:]
            for n, a in zip(name_parts, amounts):
                row['%s %s' % (name, n)] = float(a) if a != 'null' else None
            return row
        return _split_cost_column

    for cost_column in cost_columns:
        df = df.apply(split_cost_column(cost_column), axis=1)
        del df[cost_column]

    print('Applying Geo Features')
    df['bundesland'] = df[df.lat.notnull()].apply(assign_geofeature(laender_index, 'GEN'), axis=1)
    df['landkreis'] = df[df.lat.notnull()].apply(assign_geofeature(kreise_index, 'GEN'), axis=1)
    df['ags'] = df[df.lat.notnull()].apply(assign_geofeature(kreise_index, 'RS'), axis=1)
    df['Web'] = df['Web'].str.replace(' ', '').str.lower()

    df.to_csv(filename, encoding='utf-8')
    return df


df = get_nursinghome_csv()

df = df.rename(columns={
    'Koordinate Latitude': 'lat',
    'Koordinate Longitude': 'lng'
})

df.head()

,Unnamed: 0,Ausstattung/Angebote: Bibliothek,Ausstattung/Angebote: Eigene Möbel,Ausstattung/Angebote: Fahrdienst,Ausstattung/Angebote: Fernseher,Ausstattung/Angebote: Garten,Ausstattung/Angebote: Hallenbad,Ausstattung/Angebote: Haustiere,Ausstattung/Angebote: Internet,Ausstattung/Angebote: Kegelbahn,Ausstattung/Angebote: Probewohnen,Ausstattung/Angebote: Sauna,Ausstattung/Angebote: Telefon,Ausstattung/Angebote: Übernachtung von Angehörigen,E-Mail,Fax,ID,Kontakt,lat,lng,Kurzzeitpflege Allgemein Härtefall Anteil Pflegekasse,Kurzzeitpflege Allgemein Härtefall Eigenanteil,Kurzzeitpflege Allgemein Härtefall Gesamtpreis,Kurzzeitpflege Allgemein Investitionskosten Maximum,Kurzzeitpflege Allgemein Investitionskosten Minimum,Kurzzeitpflege Allgemein Pflegestufe 1 Anteil Pflegekasse,Kurzzeitpflege Allgemein Pflegestufe 1 Eigenanteil,Kurzzeitpflege Allgemein Pflegestufe 1 Gesamtpreis,Kurzzeitpflege Allgemein Pflegestufe 2 Anteil Pflegekasse,Kurzzeitpflege Allgemein Pflegestufe 2 Eigenanteil,Kurzzeitpflege Allgemein Pflegestufe 2 Gesamtpreis,Kurzzeitpflege Allgemein Pflegestufe 3 Anteil Pflegekasse,Kurzzeitpflege Allgemein Pflegestufe 3 Eigenanteil,Kurzzeitpflege Allgemein Pflegestufe 3 Gesamtpreis,Kurzzeitpflege Beatmungspflichtig Härtefall Anteil Pflegekasse,Kurzzeitpflege Beatmungspflichtig Härtefall Eigenanteil,Kurzzeitpflege Beatmungspflichtig Härtefall Gesamtpreis,Kurzzeitpflege Beatmungspflichtig Investitionskosten Maximum,Kurzzeitpflege Beatmungspflichtig Investitionskosten Minimum,Kurzzeitpflege Beatmungspflichtig Pflegestufe 1 Anteil Pflegekasse,Kurzzeitpflege Beatmungspflichtig Pflegestufe 1 Eigenanteil,Kurzzeitpflege Beatmungspflichtig Pflegestufe 1 Gesamtpreis,Kurzzeitpflege Beatmungspflichtig Pflegestufe 2 Anteil Pflegekasse,Kurzzeitpflege Beatmungspflichtig Pflegestufe 2 Eigenanteil,Kurzzeitpflege Beatmungspflichtig Pflegestufe 2 Gesamtpreis,Kurzzeitpflege Beatmungspflichtig Pflegestufe 3 Anteil Pflegekasse,Kurzzeitpflege Beatmungspflichtig Pflegestufe 3 Eigenanteil,Kurzzeitpflege Beatmungspflichtig Pflegestufe 3 Gesamtpreis,Kurzzeitpflege Demenzkrank Härtefall Anteil Pflegekasse,Kurzzeitpflege Demenzkrank Härtefall Eigenanteil,Kurzzeitpflege Demenzkrank Härtefall Gesamtpreis,Kurzzeitpflege Demenzkrank Investitionskosten Maximum,Kurzzeitpflege Demenzkrank Investitionskosten Minimum,Kurzzeitpflege Demenzkrank Pflegestufe 1 Anteil Pflegekasse,Kurzzeitpflege Demenzkrank Pflegestufe 1 Eigenanteil,Kurzzeitpflege Demenzkrank Pflegestufe 1 Gesamtpreis,Kurzzeitpflege Demenzkrank Pflegestufe 2 Anteil Pflegekasse,Kurzzeitpflege Demenzkrank Pflegestufe 2 Eigenanteil,Kurzzeitpflege Demenzkrank Pflegestufe 2 Gesamtpreis,Kurzzeitpflege Demenzkrank Pflegestufe 3 Anteil Pflegekasse,Kurzzeitpflege Demenzkrank Pflegestufe 3 Eigenanteil,Kurzzeitpflege Demenzkrank Pflegestufe 3 Gesamtpreis,Kurzzeitpflege Wachkoma Härtefall Anteil Pflegekasse,Kurzzeitpflege Wachkoma Härtefall Eigenanteil,Kurzzeitpflege Wachkoma Härtefall Gesamtpreis,Kurzzeitpflege Wachkoma Investitionskosten Maximum,Kurzzeitpflege Wachkoma Investitionskosten Minimum,Kurzzeitpflege Wachkoma Pflegestufe 1 Anteil Pflegekasse,Kurzzeitpflege Wachkoma Pflegestufe 1 Eigenanteil,Kurzzeitpflege Wachkoma Pflegestufe 1 Gesamtpreis,Kurzzeitpflege Wachkoma Pflegestufe 2 Anteil Pflegekasse,Kurzzeitpflege Wachkoma Pflegestufe 2 Eigenanteil,Kurzzeitpflege Wachkoma Pflegestufe 2 Gesamtpreis,Kurzzeitpflege Wachkoma Pflegestufe 3 Anteil Pflegekasse,Kurzzeitpflege Wachkoma Pflegestufe 3 Eigenanteil,Kurzzeitpflege Wachkoma Pflegestufe 3 Gesamtpreis,Nachtpflege Allgemein Investitionskosten Maximum,Nachtpflege Allgemein Investitionskosten Minimum,Nachtpflege Allgemein Pflegestufe 1 Anteil Pflegekasse,Nachtpflege Allgemein Pflegestufe 1 Eigenanteil,Nachtpflege Allgemein Pflegestufe 1 Gesamtpreis,Nachtpflege Allgemein Pflegestufe 2 Anteil Pflegekasse,Nachtpflege Allgemein Pflegestufe 2 Eigenanteil,Nachtpflege Allgemein Pflegestufe 2 Gesamtpreis,Nachtpflege Allgemein Pflegestufe 3 Anteil Pflegekasse,Nac

## What’s the county with the most expensive nursing homes? (Pflegeklasse 1, 2, 3 + UV / jeweils für Dauerpflege und Kurzzeitpflege)

In [38]:
def get_top_10_by_value(value, index='landkreis', top=10, asc=False):
    print('## ' + value)
    print(df.pivot_table(index=index, values=value, aggfunc=np.mean).sort_values(ascending=asc).head(10))
    print('-' * 20)

columns = [c for c in df.columns if 'Vollstationär Allgemein' in c and 'Gesamtpreis' in c]
for c in columns:
    get_top_10_by_value(c)

## Vollstationär Allgemein Härtefall Gesamtpreis
landkreis
Stuttgart                      4186.301471
St. Wendel                     4152.600000
Böblingen                      4128.381081
Steinfurt                      4090.200000
Landau in der Pfalz            4061.500000
Eifelkreis Bitburg-Prüm        4054.800000
Trier                          4053.200000
Neunkirchen                    4030.617391
Regionalverband Saarbrücken    4026.273333
Ludwigsburg                    4023.319565
Name: Vollstationär Allgemein Härtefall Gesamtpreis, dtype: float64
--------------------
## Vollstationär Allgemein Pflegestufe 1 Gesamtpreis
landkreis
Siegen-Wittgenstein    2788.850000
Kaufbeuren             2716.140000
Köln                   2705.416484
Krefeld                2653.834615
München                2631.289286
St. Wendel             2614.600000
Remscheid              2614.480000
Mülheim an der Ruhr    2608.263158
Steinfurt              2598.656897
Viersen                2597.100000
Name: Vol

In [35]:
columns = [c for c in df.columns if 'Kurzzeitpflege Allgemein' in c and 'Gesamtpreis' in c]
for c in columns:
    get_top_10_by_value(c)

## Kurzzeitpflege Allgemein Härtefall Gesamtpreis
landkreis
Cloppenburg                    4758.600000
Neckar-Odenwald-Kreis          4746.300000
Osnabrück                      4493.700000
Reutlingen                     4398.525000
Sigmaringen                    4219.500000
Würzburg                       4173.900000
Neunkirchen                    4171.125000
Stuttgart                      4104.750000
Regionalverband Saarbrücken    4082.817391
Rems-Murr-Kreis                4043.127273
Name: Kurzzeitpflege Allgemein Härtefall Gesamtpreis, dtype: float64
--------------------
## Kurzzeitpflege Allgemein Pflegestufe 1 Gesamtpreis
landkreis
Neckar-Odenwald-Kreis    4746.300000
Heidelberg               3496.800000
Würzburg                 3060.900000
München                  2952.900000
Fürstenfeldbruck         2910.600000
Berchtesgadener Land     2845.200000
Berlin                   2766.763636
Ravensburg               2741.340000
Reutlingen               2731.140000
Sigmaringen            

## What’s the county with the cheapest nursing homes? (Pflegeklasse 1, 2, 3 + UV / jeweils für Dauerpflege und Kurzzeitpflege)

In [39]:
columns = [c for c in df.columns if 'Vollstationär Allgemein' in c and 'Gesamtpreis' in c]
for c in columns:
    get_top_10_by_value(c, asc=True)

## Vollstationär Allgemein Härtefall Gesamtpreis
landkreis
Wilhelmshaven            2727.521053
Unstrut-Hainich-Kreis    2825.385000
Osterode am Harz         2860.848387
Uelzen                   2861.466667
Holzminden               2866.248000
Peine                    2871.912500
Rostock                  2872.730769
Lüneburg                 2874.618182
Schaumburg               2875.104878
Ludwigslust-Parchim      2881.008333
Name: Vollstationär Allgemein Härtefall Gesamtpreis, dtype: float64
--------------------
## Vollstationär Allgemein Pflegestufe 1 Gesamtpreis
landkreis
Unstrut-Hainich-Kreis    1619.580000
Dresden                  1623.621429
Chemnitz                 1641.193548
Mittelsachsen            1648.406897
Leipzig                  1651.989320
Rostock                  1655.511864
Ludwigslust-Parchim      1657.708333
Mansfeld-Südharz         1658.545455
Zwickau                  1660.189655
Vogtlandkreis            1664.175000
Name: Vollstationär Allgemein Pflegestufe 1 Gesam

In [40]:
columns = [c for c in df.columns if 'Kurzzeitpflege Allgemein' in c and 'Gesamtpreis' in c]
for c in columns:
    get_top_10_by_value(c, asc=True)

## Kurzzeitpflege Allgemein Härtefall Gesamtpreis
landkreis
Mannheim                 2136.600000
Herzogtum Lauenburg      2545.053659
Rendsburg-Eckernförde    2553.117647
Steinburg                2587.977778
Oldenburg                2592.300000
Segeberg                 2596.535714
Ostholstein              2605.654286
Schleswig-Flensburg      2659.555102
Nordfriesland            2670.171429
Dithmarschen             2678.334375
Name: Kurzzeitpflege Allgemein Härtefall Gesamtpreis, dtype: float64
--------------------
## Kurzzeitpflege Allgemein Pflegestufe 1 Gesamtpreis
landkreis
Oldenburg                1567.500000
Unstrut-Hainich-Kreis    1619.580000
Vogtlandkreis            1653.042857
Friesland                1661.400000
Nordwestmecklenburg      1673.130000
Saalfeld-Rudolstadt      1685.333333
Gotha                    1685.966667
Kyffhäuserkreis          1688.841176
Greiz                    1691.175000
Wittmund                 1694.100000
Name: Kurzzeitpflege Allgemein Pflegestufe 1 G

In [118]:
diligent(df[['lat', 'lng']])

Check,Dataframe,lat,lng
Data Type,,…,…
Count NaN,,…,…
Count Zeroes,,…,…
Detect suspicious values,,…,…
Detect repeated digits,,…,…
Susipicous dataframe length,…,,
Duplicate rows,…,,
Duplicate values,,…,…
Possibly numeric,,…,…
Nelson Rule 1,,…,…


## What’s the county with the most expensive food in nursing homes? (Unterkunft und Verpflegung / Dauerpflege und Kurzzeitpflege)


## What’s the county with the cheapest food in nursing homes? (Unterkunft und Verpflegung / Dauerpflege und Kurzzeitpflege)


### These questions related to cost can also be calculated compared to the the gross domestic product of the counties / states. It can also be calculated compared to the annual income of the people living in these counties / states.

In [105]:
def get_income_df(sheet='PEK je Einwohner'):
    income_df = pd.read_excel('data/VGR_KreisergebnisseBand3_5820008147005.xlsx', sheetname=sheet, skiprows=4)
    income_df = income_df.rename(columns={
            'Regional-schlüssel': 'ags'
        })
    income_df = income_df[~income_df['ags'].isnull()]
    income_df['ags'] = income_df['ags'].apply(lambda x: str(int(x)).zfill(5))

    income_years = list(range(2000, 2014))
    id_vars = ['ags', 'Gebietseinheit']
    income_df = pd.melt(income_df[income_years + id_vars], id_vars=id_vars, var_name='year', value_name=sheet)
    income_df[sheet] = pd.to_numeric(income_df[sheet], errors='coerce')
    income_df = income_df.set_index('ags')
    return income_df

In [123]:
income_df = get_income_df()
income_df[income_df.year == 2013].sort_values(by='PEK je Einwohner')

,Gebietseinheit,year,PEK je Einwohner
ags,,,
15087,"Mansfeld-Südharz, Landkreis",2013,15278.0
14626,"Görlitz, Landkreis",2013,15390.0
04012,"Bremerhaven, Kreisfreie Stadt",2013,15467.0
15089,Salzlandkreis,2013,15846.0
12066,"Oberspreewald-Lausitz, Landkreis",2013,15924.0
12073,"Uckermark, Landkreis",2013,15932.0
13075,"Vorpommern-Greifswald, Landkreis",2013,15956.0
16065,Kyffhäuserkreis,2013,15986.0
12051,"Brandenburg an der Havel, Kreisfreie Stadt",2013,16027.0


In [87]:
df = df.merge(income_df[income_df.year == 2013][['PEK je Einwohner']], left_on='ags', right_index=True)

In [88]:
columns = [c for c in df.columns if 'Vollstationär Allgemein' in c and 'Gesamtpreis' in c]
for c in columns:
    df['%s Ratio PEK/Einwohner' % c] = df[c] / df['PEK je Einwohner']

In [90]:
columns = [c for c in df.columns if 'Vollstationär Allgemein' in c and 'Gesamtpreis' in c and 'Ratio PEK/Einwohner' in c]
for c in columns:
    get_top_10_by_value(c)

## Vollstationär Allgemein Härtefall Gesamtpreis Ratio PEK/Einwohner
landkreis
Bremerhaven                    0.220120
Görlitz                        0.201192
Pirmasens                      0.195452
Uckermark                      0.195260
Offenbach am Main              0.194736
Merzig-Wadern                  0.194325
Regionalverband Saarbrücken    0.192075
Nordhausen                     0.191999
Oberspreewald-Lausitz          0.191347
Neunkirchen                    0.191079
Name: Vollstationär Allgemein Härtefall Gesamtpreis Ratio PEK/Einwohner, dtype: float64
--------------------
## Vollstationär Allgemein Pflegestufe 1 Gesamtpreis Ratio PEK/Einwohner
landkreis
Duisburg                       0.132934
Gelsenkirchen                  0.130309
Bremerhaven                    0.125933
Herne                          0.123669
Uckermark                      0.119633
Oberhausen                     0.118113
Hamm                           0.117871
Merzig-Wadern                  0.117259
Pirmasens

In [99]:
df[df['landkreis'] == 'Bremerhaven']

,Unnamed: 0,Ausstattung/Angebote: Bibliothek,Ausstattung/Angebote: Eigene Möbel,Ausstattung/Angebote: Fahrdienst,Ausstattung/Angebote: Fernseher,Ausstattung/Angebote: Garten,Ausstattung/Angebote: Hallenbad,Ausstattung/Angebote: Haustiere,Ausstattung/Angebote: Internet,Ausstattung/Angebote: Kegelbahn,Ausstattung/Angebote: Probewohnen,Ausstattung/Angebote: Sauna,Ausstattung/Angebote: Telefon,Ausstattung/Angebote: Übernachtung von Angehörigen,E-Mail,Fax,ID,Kontakt,lat,lng,Kurzzeitpflege Allgemein Härtefall Anteil Pflegekasse,Kurzzeitpflege Allgemein Härtefall Eigenanteil,Kurzzeitpflege Allgemein Härtefall Gesamtpreis,Kurzzeitpflege Allgemein Investitionskosten Maximum,Kurzzeitpflege Allgemein Investitionskosten Minimum,Kurzzeitpflege Allgemein Pflegestufe 1 Anteil Pflegekasse,Kurzzeitpflege Allgemein Pflegestufe 1 Eigenanteil,Kurzzeitpflege Allgemein Pflegestufe 1 Gesamtpreis,Kurzzeitpflege Allgemein Pflegestufe 2 Anteil Pflegekasse,Kurzzeitpflege Allgemein Pflegestufe 2 Eigenanteil,Kurzzeitpflege Allgemein Pflegestufe 2 Gesamtpreis,Kurzzeitpflege Allgemein Pflegestufe 3 Anteil Pflegekasse,Kurzzeitpflege Allgemein Pflegestufe 3 Eigenanteil,Kurzzeitpflege Allgemein Pflegestufe 3 Gesamtpreis,Kurzzeitpflege Beatmungspflichtig Härtefall Anteil Pflegekasse,Kurzzeitpflege Beatmungspflichtig Härtefall Eigenanteil,Kurzzeitpflege Beatmungspflichtig Härtefall Gesamtpreis,Kurzzeitpflege Beatmungspflichtig Investitionskosten Maximum,Kurzzeitpflege Beatmungspflichtig Investitionskosten Minimum,Kurzzeitpflege Beatmungspflichtig Pflegestufe 1 Anteil Pflegekasse,Kurzzeitpflege Beatmungspflichtig Pflegestufe 1 Eigenanteil,Kurzzeitpflege Beatmungspflichtig Pflegestufe 1 Gesamtpreis,Kurzzeitpflege Beatmungspflichtig Pflegestufe 2 Anteil Pflegekasse,Kurzzeitpflege Beatmungspflichtig Pflegestufe 2 Eigenanteil,Kurzzeitpflege Beatmungspflichtig Pflegestufe 2 Gesamtpreis,Kurzzeitpflege Beatmungspflichtig Pflegestufe 3 Anteil Pflegekasse,Kurzzeitpflege Beatmungspflichtig Pflegestufe 3 Eigenanteil,Kurzzeitpflege Beatmungspflichtig Pflegestufe 3 Gesamtpreis,Kurzzeitpflege Demenzkrank Härtefall Anteil Pflegekasse,Kurzzeitpflege Demenzkrank Härtefall Eigenanteil,Kurzzeitpflege Demenzkrank Härtefall Gesamtpreis,Kurzzeitpflege Demenzkrank Investitionskosten Maximum,Kurzzeitpflege Demenzkrank Investitionskosten Minimum,Kurzzeitpflege Demenzkrank Pflegestufe 1 Anteil Pflegekasse,Kurzzeitpflege Demenzkrank Pflegestufe 1 Eigenanteil,Kurzzeitpflege Demenzkrank Pflegestufe 1 Gesamtpreis,Kurzzeitpflege Demenzkrank Pflegestufe 2 Anteil Pflegekasse,Kurzzeitpflege Demenzkrank Pflegestufe 2 Eigenanteil,Kurzzeitpflege Demenzkrank Pflegestufe 2 Gesamtpreis,Kurzzeitpflege Demenzkrank Pflegestufe 3 Anteil Pflegekasse,Kurzzeitpflege Demenzkrank Pflegestufe 3 Eigenanteil,Kurzzeitpflege Demenzkrank Pflegestufe 3 Gesamtpreis,Kurzzeitpflege Wachkoma Härtefall Anteil Pflegekasse,Kurzzeitpflege Wachkoma Härtefall Eigenanteil,Kurzzeitpflege Wachkoma Härtefall Gesamtpreis,Kurzzeitpflege Wachkoma Investitionskosten Maximum,Kurzzeitpflege Wachkoma Investitionskosten Minimum,Kurzzeitpflege Wachkoma Pflegestufe 1 Anteil Pflegekasse,Kurzzeitpflege Wachkoma Pflegestufe 1 Eigenanteil,Kurzzeitpflege Wachkoma Pflegestufe 1 Gesamtpreis,Kurzzeitpflege Wachkoma Pflegestufe 2 Anteil Pflegekasse,Kurzzeitpflege Wachkoma Pflegestufe 2 Eigenanteil,Kurzzeitpflege Wachkoma Pflegestufe 2 Gesamtpreis,Kurzzeitpflege Wachkoma Pflegestufe 3 Anteil Pflegekasse,Kurzzeitpflege Wachkoma Pflegestufe 3 Eigenanteil,Kurzzeitpflege Wachkoma Pflegestufe 3 Gesamtpreis,Nachtpflege Allgemein Investitionskosten Maximum,Nachtpflege Allgemein Investitionskosten Minimum,Nachtpflege Allgemein Pflegestufe 1 Anteil Pflegekasse,Nachtpflege Allgemein Pflegestufe 1 Eigenanteil,Nachtpflege Allgemein Pflegestufe 1 Gesamtpreis,Nachtpflege Allgemein Pflegestufe 2 Anteil Pflegekasse,Nachtpflege Allgemein Pflegestufe 2 Eigenanteil,Nachtpflege Allgemein Pflegestufe 2 Gesamtpreis,Nachtpflege Allgemein Pflegestufe 3 Anteil Pflegekasse,Nac

In [115]:
df[df['ags'] == '03101']['Vollstationär Allgemein Pflegestufe 1 Gesamtpreis'].mean()

1867.7499999999998

In [ ]:
df

In [111]:
income_df.sort_values('PEK je Einwohner')

,Gebietseinheit,year,PEK je Einwohner
ags,,,
15087,"Mansfeld-Südharz, Landkreis",2000,10704.0
15089,Salzlandkreis,2000,10848.0
14626,"Görlitz, Landkreis",2000,10884.0
15082,"Anhalt-Bitterfeld, Landkreis",2000,10895.0
12066,"Oberspreewald-Lausitz, Landkreis",2000,10963.0
16065,Kyffhäuserkreis,2000,11049.0
15087,"Mansfeld-Südharz, Landkreis",2001,11069.0
15087,"Mansfeld-Südharz, Landkreis",2002,11070.0
15089,Salzlandkreis,2002,11114.0
